In [1]:
#imports
import faiss
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from typing import List
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

c:\Program Files\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#OPTIONS (used for finding the best way to save the chat to memory after each session)
# "whole_chat" or "individual_messages" or "pairs"
save_chat_to_memory_method = "whole_chat" 
# save_chat_to_memory_method = "individual_messages"
# save_chat_to_memory_method = "pairs"

In [3]:
#create index (aka create a database for vectors)
index = faiss.IndexFlatL2(384) #384 is the size of the vectors

In [4]:
#load embedding model and corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [5]:
#tokenise plaintext (aka slice up text in small pieces and convert those small pieces to numerical data)
def get_tokenised_sections(texts):
    tokenised_sections = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    return tokenised_sections


#tokenise each text in the array of texts in batches of 16
# def get_tokenised_sections(texts):
#     tokenised_sections = []
#     for i in range(0, len(texts), 16):
#         batch = texts[i:i+16]
#         tokenised_batch = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
#         tokenised_sections.append(tokenised_batch)
#     return tokenised_sections
    

In [6]:
#embed the texts using the tokens (aka convert the numerical data to vectors that represent the semantics of the text)
def get_embeddings(tokenised_sections):
    with torch.no_grad():
        model_output = model(**tokenised_sections)
        # Use the embedding of the [CLS] token (first token) for each input
        embeddings = model_output.last_hidden_state[:, 0, :].detach().cpu().numpy()
    return embeddings


# def get_embeddings(tokenised_sections):
#     embeddings = []
#     for section in tokenised_sections:
#         with torch.no_grad():
#             model_output = model(**section)
#             embeddings.append(model_output.last_hidden_state[:,0,:].detach().cpu().numpy())
#     return torch.cat(embeddings).numpy()


# def get_embeddings(texts):
#     tokenised_sections = get_tokenised_sections(texts)
#     embeddings = []
#     for tokenised_batch in tokenised_sections:
#         with torch.no_grad():
#             model_output = model(**tokenised_batch)
#             embeddings.append(model_output.last_hidden_state[:,0,:].detach().cpu().numpy())
#     return torch.cat(embeddings).numpy()

In [7]:
#test different things: test memory of whole chat dialog vs memory of each individual message vs memory of pairs of question and answer
#  
#putting whole chat as a memory entry
def add_to_index_wholechat(chat):
    combined_chat = ""
    for i, message in enumerate(chat):
        if i % 2 == 0: #then it is from the agent
            combined_chat += "<START OF AGENT MESSAGE>" + message + "<END OF AGENT MESSAGE>"
        else:
            combined_chat += "<START OF USER MESSAGE>" + message + "<END OF USER MESSAGE>"
        
    tokenised_chat = get_tokenised_sections([combined_chat])
    embeddings = get_embeddings(tokenised_chat)
    index.add(embeddings)

#putting each individual message as a memory entry
def add_to_index_individual_messages(chat):
    for i, message in enumerate(chat):
        if i % 2 == 0:
            tokenised_message = get_tokenised_sections(["<START OF AGENT MESSAGE>" + message + "<END OF AGENT MESSAGE>"])
        else:
            tokenised_message = get_tokenised_sections(["<START OF USER MESSAGE>" + message + "<END OF USER MESSAGE>"])
        embeddings = get_embeddings(tokenised_message)
        index.add(embeddings)

#putting each pair of user message and agent message as a memory entry
def add_to_index_pairs(chat):
    for i in range(1, len(chat), 2):
        tokenised_pair = get_tokenised_sections(["<START OF USER MESSAGE>" + chat[i] + "<END OF USER MESSAGE>", "<START OF AGENT MESSAGE>" + chat[i+1] + "<END OF AGENT MESSAGE>"])
        embeddings = get_embeddings(tokenised_pair)
        index.add(embeddings)


In [8]:
#add individual messages to index (can be used for metadata or important things the user wants the bot to remember for the current session.
#  this includes instructions for the session, user preferences, etc.)
def add_to_index(data):
    tokenised_data = get_tokenised_sections([data])
    embeddings = get_embeddings(tokenised_data)
    index.add(embeddings)

In [9]:
#similarity search
def search(query, k):
    tokenised_query = get_tokenised_sections([query])
    query_embedding = get_embeddings(tokenised_query)
    D, I = index.search(query_embedding, k)
    return D, I

In [10]:
#create tool that can be called by the llm to fetch data from memory
@tool
def fetch_From_Memory(query):
    """
    Fetch data from memory that can be used to generate a response to the user
    query: any string you think will have the highest similarity to the data you want to fetch
    return: the data that has the highest similarity to the query
    """
    print("fetching from memory: ", query)
    D, I = search(query, 10)
    return D, I
    

In [11]:
#create tool that can be called by the llm to save data to memory
@tool
def save_data_to_memory(data):
    """
    Save data to memory that can be fetched later to generate a response to the user. 
    call this function when you want to save important data like user preferences, user instructions, user personal information.
    data: any string you want to save to memory. please format it in a way that it can be easily fetched later.
    """
    print("saving data to memory: ", data)
    add_to_index(data)

In [12]:
#used after the session ends to save the chat to memory for future sessions. This is not done by the llm but automatically after each session
def save_chat_to_memory(chat):
    if save_chat_to_memory_method == "whole_chat":
        add_to_index_wholechat(chat)
    elif save_chat_to_memory_method == "individual_messages":
        add_to_index_individual_messages(chat)
    elif save_chat_to_memory_method == "pairs":
        add_to_index_pairs(chat)
    else:
        print("Invalid save_chat_to_memory_method: " + save_chat_to_memory_method)

        

In [ ]:
# #without tools
# llm = ChatOllama(
#     model="mistral",
#     temperature=0,
# )

#with tools
llm = ChatOllama(
    model="mistral",
    temperature=0,
).bind_tools([fetch_From_Memory, save_data_to_memory])

In [24]:
#save the chat so it can be put into the memory after the session ends
chat_messages = []

In [25]:
############################################
### TESTING WITH SOME RANDOM MEMORY DATA ###
############################################

user_data = "name: Jordy, age: 21, very enthousiastic, prefers reading practice questions, favourite colour: green"
user_instructions = "jordy wants to practice translating sentences about school from chinese to english, he wants you to give him feedback on his translations and also provice a new sentence after each feedback"
save_data_to_memory(user_data)
save_data_to_memory(user_instructions)

user_data = "name: John, age: 25, very calm, prefers reading books"
user_instructions = "john wants to practice translating sentences about animals from chinese to english, he wants to have conversations with you in chinese, his level is quite advanced, he has studied for 3 years already, last conversation he has held a conversation with you in chinese about studying abroad"
save_data_to_memory(user_data)
save_data_to_memory(user_instructions)

user_data = "name: 玛丽, age: 30, very energetic, prefers reading stories"
user_instructions = "玛丽 is a chinese teacher, she is preparing for a class about tones, she wants you to make example sentences about the weather specifying the tones of each character"
save_data_to_memory(user_data)
save_data_to_memory(user_instructions)

############################################
### TESTING WITH SOME RANDOM MEMORY DATA ###
############################################

saving data to memory:  name: Jordy, age: 21, very enthousiastic, prefers reading practice questions, favourite colour: green
saving data to memory:  jordy wants to practice translating sentences about school from chinese to english, he wants you to give him feedback on his translations and also provice a new sentence after each feedback
saving data to memory:  name: John, age: 25, very calm, prefers reading books
saving data to memory:  john wants to practice translating sentences about animals from chinese to english, he wants to have conversations with you in chinese, his level is quite advanced, he has studied for 3 years already, last conversation he has held a conversation with you in chinese about studying abroad
saving data to memory:  name: 玛丽, age: 30, very energetic, prefers reading stories
saving data to memory:  玛丽 is a chinese teacher, she is preparing for a class about tones, she wants you to make example sentences about the weather specifying the tones of each character

In [26]:
#send a message to the llm
def send_message(message):
    messages = [
    ("system", """YOU ARE A CONVERSATIONAL AGENT WHOSE TASK IT IS TO HELP PEOPLE WITH STUDYING LANGUAGES. 
     YOUR NAME IS AILLP.
      YOU HAVE ACCESS TO A MEMORY THAT YOU CAN USE TO FETCH FROM PAST SESSIONS AND WRITE USER INFO TO REMEMBER.
      THE CONTENTS OF THE MEMORY IS ALL FROM THE USER AND THEIR PREFERENCES OR THEIR INSTRUCTIONS OR THEIR INFO OR THEIR PAST SESSIONS WITH YOU.
      EACH TIME YOU START TALKING TO A NEW USER, YOU SHOULD FETCH FROM THE MEMORY TO SEE IF YOU HAVE ANY INFO ABOUT THE USER SUCH AS THEIR NAME, AGE, PREFERENCES, INSTRUCTIONS, OR PAST SESSIONS.
      DON'T MENTION ANYTHING ABOUT THE TOOLS OR YOUR MEMORY TO THE USER.
      IF YOU DON'T USE YOUR MEMORY THEN JUST ANSWER THE USERS QUESTION WITHOUT SAYING ANYTHING ABOUT THE MEMORY."""),
    ("user", message),
    ]
    chat_messages.append(messages)
    response = llm.invoke(messages)
    chat_messages.append(response)
    return response

# #stream output from llm (TODO LATER)
# def stream_messages(messages):
#     messages = [
#     ("system", "if needed you can try to fetch from memory using the tools provided. It is not necessary to use the tools. DON'T MENTION ANYTHING ABOUT THE TOOLS TO THE USER."),
#     ("user", "Return the words Hello World!"),
#     ]
#     for chunk in llm.stream(messages):
#         print(chunk.text(), end="")
    

#testing phase 
response = send_message("hey, im John, i want to practice chinese, can we continue from where we left of last time?")
print(response.content)

ResponseError: model "qwen2.5" not found, try pulling it first